In [1]:
from arcgis.gis import GIS, Item
import os
from multiprocessing import Process, Queue

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')
sys.path.append('../')

import reach_tools
import hydrology
import config

gis = GIS(username=config.arcgis_username, password=config.arcgis_password)

url_reach_line = config.url_reach_line
url_reach_centroid = config.url_reach_centroid
url_reach_points = config.url_reach_points

lyr_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

reach_id = 2127

%load_ext autoreload
%autoreload 2

In [2]:
reach_id_fs = lyr_centroid.query(out_fields='reach_id', result_record_count=20, return_geometry=False)
reach_id_lst = [int(feat.attributes['reach_id']) for feat in reach_id_fs.features]
print(reach_id_lst[:20])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [14]:
for reach_id in reach_id_lst:
    
    reach = reach_tools.Reach.get_from_aw(reach_id)
    reach.snap_putin_and_takeout_and_trace()

    cent_resp = lyr_centroid.update_reach(reach)

    line_resp = lyr_line.update_reach(reach)

    putin_resp = lyr_points.query(f"reach_id = '{reach_id}' AND point_type = 'access' AND subtype = 'putin'", return_ids_only=True)['objectIds']
    if len(putin_resp):
        oid_putin = putin_resp[0]
        putin_feature = reach.putin.as_feature
        putin_feature.attributes['OBJECTID'] = oid_putin
        lyr_points.edit_features(updates=[putin_feature])
    else:
        lyr_points.edit_features(adds=[putin_feature])

    
    takeout_resp = lyr_points.query(f"reach_id = '{reach_id}' AND point_type = 'access' AND subtype = 'takeout'", return_ids_only=True)['objectIds']
    if len(takeout_resp):
        oid_takeout = takeout_resp[0]
        takeout_feature = reach.takeout.as_feature
        takeout_feature.attributes['OBJECTID'] = oid_takeout
        lyr_points.edit_features(updates=[takeout_feature])
    else:
        lyr_points.edit_features(adds=[takeout_feature])
        
    print(f'success: {reach.reach_id, reach.river_name, reach.reach_name}')

success: ('1', 'Alsek', 'Haines Junction to Dry Bay')
success: ('2', 'Aniakchak', 'Surprise Lake to Aniakchak Bay')
success: ('3', 'Browns Creek', 'Browns Creek Canyon')
success: ('4', 'Canyon Creek', 'Seward Highwy at outlet of Lower Summit Lake to Seward Hwy at Hope Junction')
success: ('5', 'Charley', 'Upper river gravel bar to Yukon River to Circle')
success: ('6', 'Chetaslina', 'Chichokna River to Copper River')
success: ('7', 'Chickaloon', '30 Mile Airstrip to Matanuska')
success: ('8', 'Chilikadrotna', 'Twin Lakes (Lake Clark Nat.Park & Preserve) to points along Mulchatna River')
success: ('9', 'Chitina', 'Tana River to Copper River')
success: ('10', 'Chitistone', 'Glacier Creek to Nizina River')
success: ('11', 'Chulitna', 'East Fork to Talkeetna')
success: ('12', 'Copper (Iliamna)', 'Lower or Upper Copper Lake to Lower Pike Lake or Intricate Bay')
success: ('13', 'Dadina', 'Bush strip to Copper River')
success: ('14', 'Delta', '1. Denali Hwy at Tangle Lakes to MP 212.5 Richard

UnboundLocalError: local variable 'status' referenced before assignment

In [7]:
self = reach_tools.Reach.get_from_aw(3067)

In [5]:
reach.abstract

In [6]:
reach.description

"This is an easy class II-III run, with the hardest rapids barely approaching class III. Most of the run winds through forests, with a few small neighborhoods and a couple of bridges. Just above the put-in is a rapid that is described as class III+, but at most flows is a simple class III drop over your choice through one of 3 slots. At about the 7 mile mark, there is a diversion dam, but it's small and there's an island just above it for easy scouting on your first run. Trees could be a hazard on this run, but the river tends to be fairly wide. To get to the put-in, take Hwy 12 to Morton. Take a left at Hwy 508 (this is Main St, but there may not be a sign). After .5 miles, you will come to the bridge over the river and you'll be able to see the optional rapid. Put in under the bridge. To reach the takeout, go another 7.5 miles to another bridge over the river. The takeout is easy to recognize from the river because of 2 drain pipes sticking out from the upstream side of the bridge. T

In [8]:
(not self.abstract or len(self.abstract) == 0) and (self.abstract and len(self.abstract) > 0)

In [11]:
(not self.abstract or len(self.abstract) == 0) and (self.description and len(self.description) > 0)

True